# Import

In [1]:
import os
import numpy as np
import random
import time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope

# Load graph

In [2]:
with tf.name_scope('Preprocessing'):
    input_im = tf.placeholder(tf.uint8, shape=[None, None, None, 3], name='input_im')
    resized_im = tf.image.resize_bilinear(tf.image.convert_image_dtype(
        tf.convert_to_tensor(input_im) ,dtype=tf.float32), [299,299], name='resized_im')
    normalized_im = tf.mul(tf.sub(resized_im, 0.5), 2.0, name='normalized_im')

with slim.arg_scope(inception_v3_arg_scope()):
    logits, end_points = inception_v3(normalized_im, num_classes=1001, is_training=False)

g = tf.get_default_graph()
saver = tf.train.Saver()

## Create own branch

In [3]:
with tf.name_scope('Own'):
    y = tf.placeholder("float")
    x = tf.reshape(g.get_tensor_by_name('InceptionV3/Logits/SpatialSqueeze:0'),(1,1001))
    w = tf.Variable(tf.random_normal([int(x.get_shape()[-1]),300], stddev=float('1e-5')), name='weights')
    b = tf.Variable(tf.random_normal([1,300]), name='bias')
    y_pred = tf.add(tf.matmul(x, w), b, name='y_pred')
    cost = tf.reduce_sum(tf.square(y-y_pred),name='cost')
    cost_summary = tf.summary.scalar('cost', cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=float('1e-5'))
    train_step = optimizer.minimize(cost,var_list=[w,b])
    
timestamp = time.strftime('%Y%m%d-%H%M', time.localtime())
tf.summary.FileWriter('./sums/'+timestamp, graph=g)

# Read data

In [4]:
# Images
image_path = '../../../data/train2014/'
image_list = os.listdir(image_path)

# Caption dictionaries
vec_dict = np.load('../../../data/word2vec_train.npy').item()

# Labels
labels = open('../../../data/ilsvrc2012_labels.txt', 'r').readlines()

# Train model

In [5]:
# Tried to follow
# https://github.com/tensorflow/models/tree/master/slim
# http://stackoverflow.com/questions/35274457/inceptionv3-and-transfer-learning-with-tensorflow/40709836#40709836
with tf.Session() as sess:
    
    # Initialize variables
    saver.restore(sess, "../../../data/inception-2016-08-28/inception_v3.ckpt")
    sess.run([w.initializer, b.initializer])
    
    # Initialize writers
    train_writer = tf.summary.FileWriter('./sums/'+timestamp+'/train/', flush_secs=20)
    test_writer = tf.summary.FileWriter('./sums/'+timestamp+'/test/', flush_secs=20)
    
    # Train
    print('TRAINING')
    for i, im in enumerate(image_list[:100]):
        try:
            # Choose one of the five captions randomly
            r = random.randrange(len(vec_dict[im]))
            y_temp = vec_dict[im][r].reshape((1,300))

            # Form input and feed through
            image = [np.array(Image.open(image_path + im))]
            train_writer.add_summary(sess.run(cost_summary,{input_im:image, y:y_temp}), i)
            sess.run(train_step,feed_dict={input_im:image, y:y_temp})
            print(i, end=' ')

    #        # Get prediction
    #        class_pred = np.argmax(sess.run(
    #            g.get_tensor_by_name('InceptionV3/Predictions/Softmax:0'),{input_im:image}))
    #        print(labels[class_pred])

            # Evaluate every 10th step
            if i % 10 == 0:
                test_image = [np.array(Image.open(image_path + image_list[-1]))]
                y_test = vec_dict[image_list[-1]][1].reshape((1,300))
                test_writer.add_summary(sess.run(cost_summary,{input_im:test_image, y:y_test}), i)
        except (IndexError, ValueError):
            print('-/{}'.format(i), end=' ')
            
    # Save
    save_path = saver.save(sess, './sums/'+timestamp+'/model.ckpt')
    print('\n\nTrained model saved to {}'.format(save_path))

TRAINING
0 1 2 3 4 5 6 7 8 9 10 11 12 -/13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 Trained model saved to ./sums/20170210-1152/model.ckpt
